In [ ]:
!pip install indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 566.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 8.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt

import torch
from torch import nn,tensor
from torch.utils.data import Dataset,DataLoader
from torchtext.vocab import build_vocab_from_iterator
from tqdm import tqdm
import re
import numpy as np
import pandas as pd
import pickle
from indicnlp.tokenize import indic_tokenize
import seaborn as sns
import copy

train_path = '/content/drive/MyDrive/Colab Notebooks/INLP Project/product_reviews/hi/hi-train.csv'
test_path ='/content/drive/MyDrive/Colab Notebooks/INLP Project/product_reviews/hi/hi-test.csv'
syn_path = '/content/drive/MyDrive/Colab Notebooks/INLP Project/product_reviews/hi/hi-test-updated.csv'
val_path = '/content/drive/MyDrive/Colab Notebooks/INLP Project/product_reviews/hi/hi-valid.csv'
forward_model_path ='/content/drive/MyDrive/Colab Notebooks/INLP Project/train embeddings/pretrained_forward_model'
backward_model_path ='/content/drive/MyDrive/Colab Notebooks/INLP Project/train embeddings/pretrained_backward_model'
word_to_ix_path = '/content/drive/MyDrive/Colab Notebooks/INLP Project/train embeddings/word_to_ix.pkl'

In [ ]:

train_df = pd.read_csv(train_path, header=None, names=['Label', 'Text'])

test_df = pd.read_csv(test_path, header=None, names=['Label', 'Text'])
syn_df = pd.read_csv(syn_path, header=None, names=['Label', 'Text'])
test_df.head()



,Label,Text
0,neutral,"मगर , इस तरफ कोई ध्यान नहीं दिया जा रहा ।"
1,negative,गेम कई बार मुश्किल मालूम देता है ।
2,negative,मल्टीप्लेयर में छोटी स्टोरीलाइन है ।
3,positive,हाई - एंड एचपी नोटबुक्स की लुक्स हमेशा बेहतरीन...
4,positive,कैमरे बनाने वाली कंपनी निकोन अपनी 1 सीरीज कैमर...


In [ ]:
syn_df.head()


,Label,Text
0,neutral,"मगर , इस तरफ कोई नज़र नहीं दी जा रही ।"
1,negative,गेम कई बार कठिन मालूम देता है ।
2,negative,मल्टीप्लेयर में लघु स्टोरीलाइन है ।
3,positive,हाई - एंड एचपी नोटबुक्स की लुक्स हमेशा उत्तम ह...
4,positive,कैमरे बनाने वाली कंपनी निकोन अपनी 1 सीरीज कैमर...


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_df['Label'] = label_encoder.fit_transform(train_df['Label'])
test_df['Label'] = label_encoder.transform(test_df['Label'])
syn_df['Label'] = label_encoder.transform(syn_df['Label'])

test_df.head()


,Label,Text
0,1,"मगर , इस तरफ कोई ध्यान नहीं दिया जा रहा ।"
1,0,गेम कई बार मुश्किल मालूम देता है ।
2,0,मल्टीप्लेयर में छोटी स्टोरीलाइन है ।
3,2,हाई - एंड एचपी नोटबुक्स की लुक्स हमेशा बेहतरीन...
4,2,कैमरे बनाने वाली कंपनी निकोन अपनी 1 सीरीज कैमर...


In [ ]:
syn_df.head()


,Label,Text
0,1,"मगर , इस तरफ कोई नज़र नहीं दी जा रही ।"
1,0,गेम कई बार कठिन मालूम देता है ।
2,0,मल्टीप्लेयर में लघु स्टोरीलाइन है ।
3,2,हाई - एंड एचपी नोटबुक्स की लुक्स हमेशा उत्तम ह...
4,2,कैमरे बनाने वाली कंपनी निकोन अपनी 1 सीरीज कैमर...


In [ ]:
class ElmoProcessor(torch.nn.Module):
    def __init__(self,vocab_size,embedding_dim,hidden_dim,wt_mat):
        super(ElmoProcessor,self).__init__()
        #self.embeddings=nn.Embedding(vocab_size,embedding_dim)
        self.lstmf1=nn.LSTM(embedding_dim,hidden_dim,num_layers=1,bidirectional=False,batch_first=True)
        self.lstmf2=nn.LSTM(hidden_dim,hidden_size=hidden_dim,bidirectional=False,batch_first=True)
        self.embedding=wt_mat.to(device)
        # self.lstmb1=nn.LSTM(embedding_dim,hidden_dim,num_layers=1,bidirectional=False,batch_first=True)
        # self.lstmb2=nn.LSTM(hidden_dim,hidden_size=hidden_dim,bidirectional=False,batch_first=True)

        self.linear=nn.Linear(hidden_dim,vocab_size)
    def forward(self,embeddingdata):
        emb=torch.tensor(self.embedding(embeddingdata.to(device)))

        emb_f=emb
        # emb_b=emb.flip(1)

        o1f1,h1f1=self.lstmf1(emb_f)
        o2f2,h2f2=self.lstmf2(o1f1)
        # o2f2=o2f2+o1f1
        # h2f2=h2f2+h1f1
        # o1b1,h1b1=self.lstmb1(emb_b)
        # o2b2,h2b2=self.lstmb2(o1b1,h1b1)
        # o2b2=o2b2+o1b1
        # h2b2=h2b2+h1b1

        # weighted_vector_emb=torch.cat([emb_f,emb_b],dim=2)
        # weighted_vector_h1=torch.cat([o1f1,o1f1],dim=2)
        # weighted_vector_h2=torch.cat([o1f1,o2f2],dim=2)
        output =self.linear(o2f2)
        return (output,o1f1,o2f2)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
# # not run

# wt_mat=torch.zeros(size=(90337,300))
# embeddings=torch.tensor(wt_mat,dtype=torch.float)
# forward_model= torch.load(forward_model_path,map_location=torch.device('cpu'))
# backward_model= torch.load(backward_model_path,map_location=torch.device('cpu'))

In [ ]:
with open(word_to_ix_path, 'rb') as file:

    word_to_ix = pickle.load(file)


In [ ]:
# # not run
# mxlen=[]
# for column in train_df[train_df.columns[1]]:
#     mxlen.append(len(column.split(' ')))
# max(mxlen)

In [ ]:

mxlen = 30
class DatafetchClassify(Dataset):
    def __init__(self,split:str):
        sequence=[]
        labels=[]
        for x in tqdm(split[split.columns[1]],desc='Generate next word prediction data'):
            if len(x)<2 or x[0]=='=':continue
            words=indic_tokenize.trivial_tokenize(x,'hi') # alternative: x.strip().split(' ')
            words=words
            indices=[word_to_ix.get(word, 1) for word in words[:mxlen]]
            sequence.append((mxlen-len(indices))*[word_to_ix['<PAD>']]+indices[:])
        for x in tqdm(split[split.columns[0]],desc="Generate label"):
            labels.append(x)
        max_seq_len=max([len(ngram) for ngram in sequence])
        self.sequence=tensor(sequence)
        self.sequence.to(device)
        self.labels=tensor(labels)
        self.labels.to(device)
    def __len__(self)->int:
        return len(self.sequence)
    def __getitem__(self,index:int):
        return self.sequence[index],self.labels[index]

In [ ]:
# # not run
# classifier_dataset=DatafetchClassify(train_df)

In [ ]:
batch_size=64

In [ ]:
# # not run
# classifer_dataloader=DataLoader(classifier_dataset,batch_size,shuffle=True)
# data,labels=next(iter(classifer_dataloader))


In [ ]:
class ElmoClassifier(torch.nn.Module):
    def __init__(self,no_classes,forward_model,backward_model,embedding_dim):
        super(ElmoClassifier,self).__init__()
        self.forward_model=forward_model
        self.embedding = forward_model.embedding
        self.backward_model=backward_model
        self.forward_model.requires_grad=False
        self.backward_model.requires_grad=False
        self.linear1=nn.Linear(100,no_classes)
        self.param0=nn.Parameter(torch.rand(1),requires_grad=True)
        self.param1=nn.Parameter(torch.rand(1),requires_grad=True)
        self.param2=nn.Parameter(torch.rand(1),requires_grad=True)
        self.bilstm=nn.LSTM(input_size=embedding_dim*2,hidden_size=50,bidirectional=True,batch_first=True)
    def forward(self,embdata):
        _,o1f1,o2f2=self.forward_model(embdata)
        _,o1b1,o2b2=self.backward_model(embdata)
        # print(embdata.shape)
        # print('\n',o1f1.shape)
        embdata = self.embedding(embdata)
        e = torch.cat([embdata, embdata.flip(1)],dim=2)
        h1=torch.cat([o1f1,o1b1.flip(1)],dim=2)
        h2=torch.cat([o2f2,o2b2.flip(1)],dim=2)
        bio,bih=self.bilstm(self.param0*e + self.param1*h1 +self.param2*h2)

        y=self.linear1(bio[:,-1,:])

        return y

In [ ]:
# # not run
# classifier=ElmoClassifier(3,forward_model,backward_model,300)

In [ ]:
# # not run

# for epoch in range(5):
#     train_loss=0
#     classifier.train()
#     for batch in tqdm(classifer_dataloader,desc="Training"):
#         x_train,y_train=batch
#         optimizer.zero_grad()

#         x_train.to(device)
#         y_train.to(device)
#         output=classifier(x_train)
#         loss=loss_fn(output.to(device),y_train.to(device))
#         loss.backward()
#         optimizer.step()
#         train_loss +=loss.item()
#     print(train_loss)

In [ ]:
classifier_eval=DatafetchClassify(test_df)

Generate label: 100%|██████████| 523/523 [00:00<00:00, 500027.58it/s]


In [ ]:
classifer_dataloader_eval=DataLoader(classifier_eval,batch_size=100,shuffle=None)
data,labels=next(iter(classifer_dataloader_eval))


In [ ]:
classifier_path = '/content/drive/MyDrive/Colab Notebooks/INLP Project/product_reviews/product_review_classifier_final'
classifier = torch.load(classifier_path, map_location=torch.device('cpu'))

classifier.to(device)
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(classifier.parameters(),1e-3)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score,recall_score
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
confusion_matrix=np.zeros((4,4))
y_true=[]
y_pred=[]
train_loss=0.0
for batch in tqdm(classifer_dataloader_eval,desc="Evaluation"):
    x_train,y_train=batch
    optimizer.zero_grad()

    x_train.to(device)
    y_train.to(device)
    output=classifier(x_train)

    _,preds=torch.max(output,dim=1)
    y_true.extend(y_train.cpu().detach().numpy())
    y_pred.extend(preds.cpu().detach().numpy())
    break

Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]<ipython-input-8-b766623ffbb4>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb=torch.tensor(self.embedding(embeddingdata.to(device)))
Evaluation:   0%|          | 0/6 [00:15<?, ?it/s]


In [ ]:
output.shape

torch.Size([100, 3])

In [ ]:
len(y_true)

100

In [ ]:
output_exp = torch.exp(output)
row_sums = torch.sum(output_exp, dim=1, keepdim=True)

# Divide each element by its row sum
output = output_exp / row_sums

In [ ]:
y_org_sentence = copy.copy(output)
y_org_sentence

tensor([[1.4500e-01, 7.5829e-01, 9.6705e-02],
        [7.1865e-01, 1.2425e-01, 1.5710e-01],
        [4.4237e-02, 1.9106e-01, 7.6470e-01],
        [1.4982e-03, 5.0873e-03, 9.9341e-01],
        [1.4704e-03, 5.4044e-03, 9.9313e-01],
        [1.7475e-02, 6.3854e-02, 9.1867e-01],
        [2.9203e-03, 9.3498e-01, 6.2104e-02],
        [3.2640e-02, 1.2221e-01, 8.4515e-01],
        [2.6688e-01, 5.9621e-02, 6.7350e-01],
        [1.1906e-03, 9.7639e-01, 2.2420e-02],
        [1.5739e-02, 9.7020e-01, 1.4060e-02],
        [1.2935e-03, 9.6891e-01, 2.9796e-02],
        [2.1833e-03, 1.3467e-01, 8.6315e-01],
        [2.8925e-03, 2.3313e-01, 7.6398e-01],
        [1.8231e-02, 8.2313e-02, 8.9946e-01],
        [2.0777e-02, 4.5873e-01, 5.2049e-01],
        [1.7107e-01, 2.5972e-01, 5.6921e-01],
        [7.5665e-04, 9.7549e-01, 2.3752e-02],
        [3.2285e-02, 1.8977e-01, 7.7795e-01],
        [1.6483e-03, 9.4030e-01, 5.8047e-02],
        [8.5204e-01, 8.0656e-02, 6.7302e-02],
        [2.4141e-03, 9.8315e-01, 1

In [ ]:
# accuracy_score(y_true,y_pred)

In [ ]:
classifier_eval_syn =DatafetchClassify(syn_df)

Generate label: 100%|██████████| 523/523 [00:00<00:00, 962959.17it/s]


In [ ]:
classifer_dataloader_eval_syn =DataLoader(classifier_eval_syn,batch_size=100,shuffle=None)
data,labels=next(iter(classifer_dataloader_eval_syn))


In [ ]:
# classifier_path = '/content/drive/MyDrive/Colab Notebooks/INLP Project/product_reviews/product_review_classifier_final'
# classifier = torch.load(classifier_path, map_location=torch.device('cpu'))

# classifier.to(device)
# loss_fn=nn.CrossEntropyLoss()
# optimizer=torch.optim.Adam(classifier.parameters(),1e-3)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score,recall_score
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
confusion_matrix=np.zeros((4,4))
y_true=[]
y_pred=[]
train_loss=0.0
for batch in tqdm(classifer_dataloader_eval_syn,desc="Evaluation"):
    x_train,y_train=batch
    optimizer.zero_grad()

    x_train.to(device)
    y_train.to(device)
    output=classifier(x_train)

    _,preds=torch.max(output,dim=1)
    y_true.extend(y_train.cpu().detach().numpy())
    y_pred.extend(preds.cpu().detach().numpy())
    break

Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]<ipython-input-8-b766623ffbb4>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb=torch.tensor(self.embedding(embeddingdata.to(device)))
Evaluation:   0%|          | 0/6 [00:09<?, ?it/s]


In [ ]:
# accuracy_score(y_true,y_pred)

In [ ]:
output.shape

torch.Size([100, 3])

In [ ]:
output_exp = torch.exp(output)
row_sums = torch.sum(output_exp, dim=1, keepdim=True)

# Divide each element by its row sum
output = output_exp / row_sums

In [ ]:
y_syn_sentence = copy.copy(output)
y_syn_sentence

tensor([[1.8545e-01, 7.5790e-01, 5.6644e-02],
        [6.8935e-01, 1.4252e-01, 1.6814e-01],
        [3.8451e-02, 2.7076e-01, 6.9079e-01],
        [1.3920e-03, 6.4844e-03, 9.9212e-01],
        [1.6820e-03, 8.7334e-03, 9.8958e-01],
        [7.4998e-03, 5.0824e-02, 9.4168e-01],
        [1.8359e-02, 3.9388e-01, 5.8776e-01],
        [2.2322e-02, 1.2583e-01, 8.5185e-01],
        [4.2467e-01, 7.4097e-02, 5.0124e-01],
        [1.1945e-03, 9.6913e-01, 2.9679e-02],
        [1.4841e-02, 9.7116e-01, 1.3996e-02],
        [1.4558e-03, 9.5048e-01, 4.8065e-02],
        [2.6082e-03, 2.3639e-01, 7.6100e-01],
        [5.9870e-03, 6.5802e-01, 3.3599e-01],
        [3.6501e-03, 8.6198e-03, 9.8773e-01],
        [1.5367e-02, 1.1755e-01, 8.6709e-01],
        [1.2095e-02, 1.8722e-01, 8.0068e-01],
        [7.3950e-04, 9.7318e-01, 2.6080e-02],
        [3.0967e-02, 2.1786e-01, 7.5118e-01],
        [1.2063e-03, 9.3059e-01, 6.8205e-02],
        [8.5335e-01, 7.9012e-02, 6.7643e-02],
        [5.4087e-03, 9.7117e-01, 2

In [ ]:
import statistics
def evaluate_embeddings(y_org_sentence, y_syn_sentence, y_true):
  differences = []
  for idx in range(len(y_true)):
    y_hat_syn = y_syn_sentence[idx][y_true[idx]]
    y_hat = y_org_sentence[idx][y_true[idx]]
    # print(y_hat_syn)
    # print(y_hat)
    diff = abs(y_hat - y_hat_syn)
    differences.append(diff)
  metric_mean =  sum(differences)/len(y_pred) #
  metric_median = statistics.median(differences)
  return metric_mean, metric_median
metric_mean, metric_median = evaluate_embeddings(y_org_sentence, y_syn_sentence, y_true)

In [ ]:
metric_mean

tensor(0.0820, grad_fn=<DivBackward0>)

In [ ]:
metric_median

tensor(0.0178, grad_fn=<DivBackward0>)